# Ollama for Mistal 7B and Langchain


| Model               | Parameters | Size  | Download               |
|---------------------|------------|-------|------------------------|
| Mistral             | 7B         | 4.1GB | ollama run mistral     |
| Llama 2             | 7B         | 3.8GB | ollama run llama2      |
| Code Llama          | 7B         | 3.8GB | ollama run codellama   |
| Llama 2 Uncensored  | 7B         | 3.8GB | ollama run llama2-uncensored |
| Llama 2 13B         | 13B        | 7.3GB | ollama run llama2:13b  |
| Llama 2 70B         | 70B        | 39GB  | ollama run llama2:70b  |
| Orca Mini           | 3B         | 1.9GB | ollama run orca-mini   |
| Vicuna              | 7B         | 3.8GB | ollama run vicuna      |


[LangChain doc](https://python.langchain.com/docs/integrations/llms/ollama)

In [1]:
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler                                  
llm = Ollama(model="mistral", 
             callback_manager = CallbackManager([StreamingStdOutCallbackHandler()]))

In [2]:
llm("Hello, my name is Ruben and just downloaded Mistral 7B.")

Hi Ruben! Welcome to the Mistral community. I'm here to help you with any questions or issues you may have regarding the installation and usage of Mistral 7B. Please feel free to ask me anything you need assistance with.

"Hi Ruben! Welcome to the Mistral community. I'm here to help you with any questions or issues you may have regarding the installation and usage of Mistral 7B. Please feel free to ask me anything you need assistance with."

In [3]:
llm("Tell me about the history of Mistral AI")


Mistral AI is a cutting-edge company that specializes in developing large language models. I am not sure when it was founded or who founded it, but I can tell you that it was established relatively recently, in 2017. Since then, it has become one of the leading companies in the field of natural language processing, thanks to its innovative technology and impressive achievements.
Mistral AI's flagship product is Mistral, a powerful and versatile language model that can be used for a wide range of tasks, such as text classification, language translation, question answering, and more. It was trained on massive amounts of data using state-of-the-art techniques, and it has achieved state-of-the-art results on many benchmark tasks.
In addition to its work on large language models, Mistral AI also focuses on other areas of natural language processing, such as dialogue systems, sentiment analysis, and text generation. The company is constantly pushing the boundaries of what is possible with la

"\nMistral AI is a cutting-edge company that specializes in developing large language models. I am not sure when it was founded or who founded it, but I can tell you that it was established relatively recently, in 2017. Since then, it has become one of the leading companies in the field of natural language processing, thanks to its innovative technology and impressive achievements.\n\nMistral AI's flagship product is Mistral, a powerful and versatile language model that can be used for a wide range of tasks, such as text classification, language translation, question answering, and more. It was trained on massive amounts of data using state-of-the-art techniques, and it has achieved state-of-the-art results on many benchmark tasks.\n\nIn addition to its work on large language models, Mistral AI also focuses on other areas of natural language processing, such as dialogue systems, sentiment analysis, and text generation. The company is constantly pushing the boundaries of what is possibl

In [4]:
llm("what was the former question?")

I am unable to provide a specific answer as I don't have context on what the previous question was. Can you please provide more details or specify what information you are looking for?

"I am unable to provide a specific answer as I don't have context on what the previous question was. Can you please provide more details or specify what information you are looking for?"

# Use Ollama Embeddings

In [5]:
from langchain.embeddings import OllamaEmbeddings
oembed = OllamaEmbeddings(base_url="http://localhost:11434", model="mistral")
oembed.embed_query("Llamas are social animals and live with others as a herd.")

[8.905773162841797,
 -0.3007132411003113,
 1.6585137844085693,
 4.14909553527832,
 2.312321424484253,
 5.652364253997803,
 1.576897382736206,
 1.8415940999984741,
 2.569943428039551,
 1.762722134590149,
 9.892632484436035,
 -4.531696319580078,
 -5.059666156768799,
 6.027589797973633,
 -6.238118648529053,
 -0.7480594515800476,
 -1.684426188468933,
 -2.5290794372558594,
 -1.994338035583496,
 -0.6451694369316101,
 6.242337226867676,
 -5.356689453125,
 -1.6293590068817139,
 0.790675699710846,
 2.1867523193359375,
 -3.6383323669433594,
 6.569645404815674,
 -1.8111851215362549,
 -3.2242400646209717,
 -0.9286804795265198,
 -2.29052472114563,
 4.043195724487305,
 -2.527665376663208,
 3.64412260055542,
 -6.08896541595459,
 2.3799631595611572,
 -0.8551969528198242,
 -0.7795040607452393,
 1.0490927696228027,
 -0.010281318798661232,
 0.6357641816139221,
 -5.319364547729492,
 -0.36242783069610596,
 -0.381481796503067,
 -2.976294755935669,
 -1.3664796352386475,
 -2.433171033859253,
 -0.5754787921905

# RAG aaplication


Let's also use local embeddings from OllamaEmbeddings and Chroma.


In [6]:
#!pip install chromadb

In [7]:
# Load web page
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.bbc.com/culture/article/20231013-the-300-year-old-japanese-method-of-upcyling")
data = loader.load()

In [8]:
# Split into chunks 
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)
all_splits = text_splitter.split_documents(data)

In [10]:
# Embed and store
from langchain.vectorstores import Chroma
from langchain.embeddings import GPT4AllEmbeddings
from langchain.embeddings import OllamaEmbeddings # We can also try Ollama embeddings
vectorstore = Chroma.from_documents(documents=all_splits,
                                    embedding=OllamaEmbeddings())

In [11]:
# Retrieve
question = "What is this article about?"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [12]:
# RAG prompt
from langchain import hub
QA_CHAIN_PROMPT = hub.pull("rlm/rag-prompt-llama")

In [13]:
# LLM
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
llm = Ollama(model="mistral",
             verbose=True,
             callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))

In [14]:
# QA chain
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

In [15]:
question = "What is this article about?"
result = qa_chain({"query": question})

The article is about sashiko, a basic form of stitching used for repairing woven fabrics. It is versatile and easy to use, making it ideal for beginners. The technique involves pulling a thread through the fabric from front to back and up again, creating rows that can be filled in to create a strong 'brick wall' of stitches. Sashiko needles are longer, allowing menders to do more than one stitch at a time. The popularity of sashiko workshops has been increasing recently due to a focus on sustainability and the desire to learn ancient crafts.

In [28]:
from langchain.schema import LLMResult
from langchain.callbacks.base import BaseCallbackHandler

class GenerationStatisticsCallback(BaseCallbackHandler):
    def on_llm_end(self, response: LLMResult, **kwargs) -> None:
        print(response.generations[0][0].generation_info)
        
callback_manager = CallbackManager([StreamingStdOutCallbackHandler(), GenerationStatisticsCallback()])

llm = Ollama(base_url="http://localhost:11434",
             model="mistral",
             verbose=True,
             callback_manager=callback_manager)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

question = "What is this article about?"
result = qa_chain({"query": question})

This article is about sashiko, a basic form of stitching used for repairing woven fabrics. It explains how to use sashiko needles, yarn, and other materials to create strong "brick wall" stitches that can be used on all types of fabrics. The article also talks about the popularity of sashiko workshops and how they are being used in various industries, including fashion, for sustainability and creativity.{'model': 'mistral', 'created_at': '2023-10-16T21:19:56.277254Z', 'response': '', 'done': True, 'context': [733, 16289, 28793, 10649, 28747, 733, 16289, 28793, 5275, 18741, 4060, 995, 460, 396, 13892, 354, 2996, 28733, 509, 28727, 2131, 9796, 28723, 5938, 272, 2296, 7769, 302, 17913, 286, 2758, 298, 4372, 272, 2996, 28723, 1047, 368, 949, 28742, 28707, 873, 272, 4372, 28725, 776, 1315, 369, 368, 949, 28742, 28707, 873, 28723, 5938, 1712, 23748, 7228, 304, 1840, 272, 4372, 3078, 864, 26364, 700, 18741, 4060, 28705, 13, 24994, 28747, 1824, 349, 456, 5447, 684, 28804, 28705, 13, 2083, 2874

In [29]:
question = "What was the last question?"
result = qa_chain({"query": question})


The last context provided was about sashiko workshops, their popularity, and the reasons behind them. The workshops are increasing in demand due to the focus on sustainability and mending in the fashion industry. They allow people to learn a beautiful ancient craft and make their clothing more bespoke and interesting. The workshops also provide a great atmosphere for like-minded individuals to meet and share their passion for sashiko.{'model': 'mistral', 'created_at': '2023-10-16T21:20:23.735643Z', 'response': '', 'done': True, 'context': [733, 16289, 28793, 10649, 28747, 733, 16289, 28793, 5275, 18741, 4060, 995, 460, 396, 13892, 354, 2996, 28733, 509, 28727, 2131, 9796, 28723, 5938, 272, 2296, 7769, 302, 17913, 286, 2758, 298, 4372, 272, 2996, 28723, 1047, 368, 949, 28742, 28707, 873, 272, 4372, 28725, 776, 1315, 369, 368, 949, 28742, 28707, 873, 28723, 5938, 1712, 23748, 7228, 304, 1840, 272, 4372, 3078, 864, 26364, 700, 18741, 4060, 28705, 13, 24994, 28747, 1824, 403, 272, 1432, 2

# Using the Hub for prompt management

In [31]:
# LLM
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
llm = Ollama(model="mistral",
             verbose=True,
             callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))

In [32]:
from langchain import hub
QA_CHAIN_PROMPT = hub.pull("rlm/rag-prompt-mistral")

# QA chain
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

In [33]:
question = "What is this article about?"
result = qa_chain({"query": question})

This article is about sashiko, a basic form of stitching commonly used for denim repair. It is considered ideal for beginners as it only requires water-soluble marking pen or pencil to draw patterns on the cloth and sashiko needles and thimble. The patch is not the most prominent feature; rather, the stitches are what make it appealing. Sashiko workshops have been increasing in popularity due to their versatility across all woven fabrics and their focus on mending for sustainability reasons. They provide a great skill to learn within the workshop atmosphere and meet like-minded people.